# Function Calling

In [1]:
import os
import re
import sys
import json
import operator
from pprint import pprint
from datetime import datetime
from dotenv import load_dotenv
from typing import Optional, Union
from langchain_community.llms.sambanova import SambaStudio, Sambaverse
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import StructuredTool
from langchain_core.tools import ToolException
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.tool import ToolMessage
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import create_sql_query_chain
from langchain.globals import set_debug

set_debug(False)

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, '..'))
repo_dir = os.path.abspath(os.path.join(kit_dir, '..'))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

load_dotenv(os.path.join(repo_dir, '.env'))

True

## Tools Definitions

### Basic tools

#### Get time

In [2]:
# tool schema
class GetTimeSchema(BaseModel):
    """Returns current date, current time or both."""

    kind: Optional[str] = Field(description='kind of information to retrieve "date", "time" or "both"')

In [3]:
# definition using @tool decorator
@tool(args_schema=GetTimeSchema)
def get_time(kind: str = 'both') -> str:
    """Returns current date, current time or both.

    Args:
        kind: date, time or both
    """
    if kind == 'date':
        date = datetime.now().strftime('%d/%m/%Y')
        return f'Current date: {date}'
    elif kind == 'time':
        time = datetime.now().strftime('%H:%M:%S')
        return f'Current time: {time}'
    else:
        date = datetime.now().strftime('%d/%m/%Y')
        time = datetime.now().strftime('%H:%M:%S')
        return f'Current date: {date}, Current time: {time}'

In [4]:
get_time.invoke({'kind': 'time'})

'Current time: 10:53:26'

In [5]:
get_time.get_input_schema().schema()

{'title': 'GetTimeSchema',
 'description': 'Returns current date, current time or both.',
 'type': 'object',
 'properties': {'kind': {'title': 'Kind',
   'description': 'kind of information to retrieve "date", "time" or "both"',
   'type': 'string'}}}

### Customized error handling tools

#### Calculator

In [6]:
# tool schema
class CalculatorSchema(BaseModel):
    """allow calculation of only basic operations: + - * and /
    with a string input expression"""

    expression: str = Field(..., description="expression to calculate, example '12 * 3'")

In [31]:
# function to use in the tool
def calculator(expression: str) -> Union[str, int, float]:
    """
    allow calculation of basic operations
    with a string input expression
    Args:
        expression: expression to calculate
    """
    ops = {
        '+': operator.add,
        '-': operator.sub,
        '*': operator.mul,
        'x': operator.mul,
        'X': operator.mul,
        '÷': operator.truediv,
        '/': operator.truediv,
    }
    tokens = re.findall(r'\d+\.?\d*|\+|\-|\*|\/|÷|x|X', expression)

    if len(tokens) == 0:
        raise ToolException(
            f"Invalid expression '{expression}', should only contain one of the following operators + - * and /"
        )

    current_value = float(tokens.pop(0))

    while len(tokens) > 0:
        # The next token should be an operator
        op = tokens.pop(0)

        # The next token should be a number
        if len(tokens) == 0:
            raise ToolException(f"Incomplete expression '{expression}'")
        try:
            next_value = float(tokens.pop(0))

        except ValueError:
            raise ToolException('Invalid number format')

        except:
            raise ToolException('Invalid operation')

        # check division by 0
        if op in ['/', '÷'] and next_value == 0:
            raise ToolException('cannot divide by 0')

        current_value = ops[op](current_value, next_value)

    result = current_value

    return result


# tool error handler
def _handle_error(error: ToolException) -> str:
    return f'The following errors occurred during Calculator tool execution: `{error.args}`'


# tool definition
calculator = StructuredTool.from_function(
    func=calculator,
    args_schema=CalculatorSchema,
    handle_tool_error=_handle_error,  # True,
)

In [32]:
calculator.invoke('7 / 0')

"The following errors occurred during Calculator tool execution: `('cannot divide by 0',)`"

In [33]:
calculator.get_input_schema().schema()

{'title': 'CalculatorSchema',
 'description': 'allow calculation of only basic operations: + - * and /\nwith a string input expression',
 'type': 'object',
 'properties': {'expression': {'title': 'Expression',
   'description': "expression to calculate, example '12 * 3'",
   'type': 'string'}},
 'required': ['expression']}

### Langchain Tools

#### Python repl

In [34]:
# tool schema
class ReplSchema(BaseModel):
    "A Python shell. Use this to execute python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...), if you need a specific module you should import it`."

    command: str = Field(..., description='python code to execute to evaluate')

In [11]:
# tool definition
python_repl = PythonREPL()
python_repl = Tool(
    name='python_repl',
    description='A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.',
    func=python_repl.run,
    args_schema=ReplSchema,
)

In [12]:
python_repl.invoke({'command': 'for i in range(0,5):\n\tprint(i)'})

Python REPL can execute arbitrary code. Use with caution.


'0\n1\n2\n3\n4\n'

In [13]:
python_repl.get_input_schema().schema()

{'title': 'ReplSchema',
 'description': 'A Python shell. Use this to execute python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...), if you need a specific module you should import it`.',
 'type': 'object',
 'properties': {'command': {'title': 'Command',
   'description': 'python code to execute to evaluate',
   'type': 'string'}},
 'required': ['command']}

#### SQL calling

##### This tools is a demonstration of how to implement an SQL query tool, for this you will need to have a fine tunned sql model, in this exaample we provide one available in Sambaverse, for fine tuning your own sql model go to [fine_tuning_sql kit](../../fine_tuning_sql/README.md)

In [14]:
# example sql query call
db_path = os.path.join(kit_dir, 'data/chinook.db')
db_uri = f'sqlite:///{db_path}'
db = SQLDatabase.from_uri(db_uri)
print(db.get_usable_table_names())
print(db.run('SELECT * FROM genres;'))

['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']
[(1, 'Rock'), (2, 'Jazz'), (3, 'Metal'), (4, 'Alternative & Punk'), (5, 'Rock And Roll'), (6, 'Blues'), (7, 'Latin'), (8, 'Reggae'), (9, 'Pop'), (10, 'Soundtrack'), (11, 'Bossa Nova'), (12, 'Easy Listening'), (13, 'Heavy Metal'), (14, 'R&B/Soul'), (15, 'Electronica/Dance'), (16, 'World'), (17, 'Hip Hop/Rap'), (18, 'Science Fiction'), (19, 'TV Shows'), (20, 'Sci Fi & Fantasy'), (21, 'Drama'), (22, 'Comedy'), (23, 'Alternative'), (24, 'Classical'), (25, 'Opera')]


In [15]:
# tool schema
class QueryDBSchema(BaseModel):
    "A database querying tool. Use this to generate sql querys and retrieve the results from a database. Input should be a natural language question to the db."

    query: str = Field(..., description='python code to execute to evaluate')

In [16]:
@tool(args_schema=QueryDBSchema)
def query_db(query):
    """A database querying tool. Use this to generate sql querys and retrieve the results from a database. Input should be a natural language question to the db."""

    fine_tunned_sql_llm = 'nsql-llama-2-7b'
    llm = Sambaverse(
        sambaverse_model_name='Numbers Station/nsql-llama-2-7b',
        model_kwargs={'max_tokens_to_generate': 256, 'select_expert': fine_tunned_sql_llm, 'process_prompt': True},
    )

    prompt = PromptTemplate.from_template(
        '{table_info}\n\n-- Using valid SQLite, answer the following questions for the tables provided above.\n\n-- {input} {top_k}\n\nSELECT'
    )
    write_query = create_sql_query_chain(llm, db, prompt=prompt)

    # print(write_query.invoke({'question': query}))

    execute_query = QuerySQLDataBaseTool(db=db)
    sql_chain = write_query | execute_query

    return sql_chain.invoke({'question': query})

In [88]:
query_db.invoke({'query': 'How many genres of music are in the chinook db'})

'[(25,)]'

### Default response tool

In [17]:
# tool schema
class ConversationalResponse(BaseModel):
    "Respond conversationally only if no other tools should be called for a given query, or if you have a final answer."

    response: str = Field(..., description='Conversational response to the user.')


ConversationalResponse.schema()

{'title': 'ConversationalResponse',
 'description': 'Respond conversationally only if no other tools should be called for a given query, or if you have a final answer.',
 'type': 'object',
 'properties': {'response': {'title': 'Response',
   'description': 'Conversational response to the user.',
   'type': 'string'}},
 'required': ['response']}

In [35]:
def get_tools_schemas(tools: Union[Tool, list] = None, default: Union[Tool, BaseModel] = None):
    if tools is None:
        pass
    elif isinstance(tools, Tool):
        tools = [tools]
    tools_schemas = []

    for tool in tools:
        tool_schema = tool.get_input_schema().schema()
        schema = {'name': tool.name, 'description': tool_schema['description'], 'properties': tool_schema['properties']}
        if 'required' in schema:
            schema['required'] = tool_schema['required']
        tools_schemas.append(schema)

    if default is not None:
        if isinstance(default, Tool):
            tool_schema = default.get_input_schema().schema()
        else:
            tool_schema = default.schema()
        schema = {
            'name': tool_schema['title'],
            'description': tool_schema['description'],
            'properties': tool_schema['properties'],
        }
        if 'required' in schema:
            schema['required'] = tool_schema['required']
        tools_schemas.append(schema)

    return tools_schemas

In [36]:
tools = [get_time, calculator, python_repl]

In [37]:
tools_schemas = get_tools_schemas(tools, default=ConversationalResponse)
tools_schemas = '\n'.join([json.dumps(tool, indent=2) for tool in tools_schemas])
pprint(tools_schemas)

('{\n'
 '  "name": "get_time",\n'
 '  "description": "Returns current date, current time or both.",\n'
 '  "properties": {\n'
 '    "kind": {\n'
 '      "title": "Kind",\n'
 '      "description": "kind of information to retrieve \\"date\\", \\"time\\" '
 'or \\"both\\"",\n'
 '      "type": "string"\n'
 '    }\n'
 '  }\n'
 '}\n'
 '{\n'
 '  "name": "calculator",\n'
 '  "description": "allow calculation of only basic operations: + - * and '
 '/\\nwith a string input expression",\n'
 '  "properties": {\n'
 '    "expression": {\n'
 '      "title": "Expression",\n'
 '      "description": "expression to calculate, example \'12 * 3\'",\n'
 '      "type": "string"\n'
 '    }\n'
 '  }\n'
 '}\n'
 '{\n'
 '  "name": "python_repl",\n'
 '  "description": "A Python shell. Use this to execute python commands. Input '
 'should be a valid python commands and expressions. If you want to see the '
 'output of a value, you should print it out with `print(...), if you need a '
 'specific module you should im

## Function Calling 

### LLM definition

In [38]:
llm = Sambaverse(
    sambaverse_model_name='Meta/Meta-Llama-3-70B-Instruct',
    model_kwargs={
        'max_tokens_to_generate': 2048,
        'select_expert': 'Meta-Llama-3-70B-Instruct',
        'process_prompt': True,
        'temperature': 0.01,
    },
)

# llm= SambaStudio(
#     streaming=True,
#     model_kwargs={
#             "max_tokens_to_generate": 2048,
#             "select_expert": "Meta-Llama-3-70B-Instruct",
#             "process_prompt": False
#         }
# )

### Tool execution

In [22]:
tools_map = {
    'get_time': get_time,
    'calculator': calculator,
    'python_repl': python_repl,
}


def execute(tools):
    tool_msg = "Tool '{name}'response: {response}"
    tools_msgs = []
    if len(tools) == 1 and tools[0]['tool'].lower() == 'conversationalresponse':
        final_answer = True
        return final_answer, tools[0]['tool_input']['response']
    for tool in tools:
        final_answer = False
        if tool['tool'].lower() != 'conversationalresponse':
            response = tools_map[tool['tool'].lower()](tool['tool_input'])
            tools_msgs.append(tool_msg.format(name=tool['tool'], response=str(response)))
    return final_answer, tools_msgs

In [23]:
def jsonFinder(input_string):
    json_pattern = re.compile(r'(\{.*\}|\[.*\])', re.DOTALL)
    # Find the first JSON structure in the string
    json_match = json_pattern.search(input_string)
    if json_match:
        json_str = json_match.group(1)
        try:
            json.loads(json_str)
        except:
            json_correction_prompt = """|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a json format corrector tool<|eot_id|><|start_header_id|>user<|end_header_id|>
            fix the following json file: {json} 
            <|eot_id|><|start_header_id|>assistant<|end_header_id|>
            fixed json: """
            json_correction_prompt_template = PromptTemplate.from_template(json_correction_prompt)
            json_correction_chain = json_correction_prompt_template | llm
            json_str = json_correction_chain.invoke(json_str)
    else:
        # implement here not finding json format parsing to json or error rising
        json_str = None
    return json_str

In [24]:
example_function_calling_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an helpful assistant and you have access to the following tools:

{tools}

You must always select one or more of the above tools and answer with only a list of JSON objects matching the following schema:

```json
[{{
  "tool": <name of the selected tool>,
  "tool_input": <parameters for the selected tool, matching the tool's JSON schema>
}}]
```

Think step by step
Do not call a tool if the input depends on another tool output you dont have yet
Do not try to answer until you get tools output, if you dont have an answer yet you can continue calling tools

<|eot_id|><|start_header_id|>user<|end_header_id|>
User: {usr_msg} 
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Assistant:"""
# You must wait to have a tools output before generating a final response, or calling other tools
function_calling_prompt_template = PromptTemplate.from_template(example_function_calling_prompt)

In [25]:
json_parsing_chain = RunnableLambda(jsonFinder) | JsonOutputParser()

In [26]:
prompt_template = {
    'tools': lambda x: tools_schemas,
    'usr_msg': RunnablePassthrough(),
} | function_calling_prompt_template

In [27]:
default_fc_chain = prompt_template | llm | json_parsing_chain

In [28]:
query = 'hi'
response_tools = default_fc_chain.invoke(query)
print(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'ConversationalResponse', 'tool_input': {'response': 'Hi! How can I assist you today?'}}]
Hi! How can I assist you today?


In [194]:
query = 'it is time to go to sleep?'
response_tools = default_fc_chain.invoke(query)
pprint(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'get_time', 'tool_input': {'kind': 'time'}}]
["Tool 'get_time'response: Current time: 11:34:10"]


In [39]:
query = 'whats is 347 min in hours and minutes?'
response_tools = default_fc_chain.invoke(query)
print(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'calculator', 'tool_input': {'expression': '347 / 60'}}]
["Tool 'calculator'response: 5.783333333333333"]


/Users/jorgep/Documents/ask_public_own/fcenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [196]:
query = "is this word is a palindrome? 'saippuakivikauppias'"
response_tools = default_fc_chain.invoke(query)
pprint(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'python_repl',
  'tool_input': {'command': "print('saippuakivikauppias' "
                            "=='saippuakivikauppias'[::-1])"}}]
["Tool 'python_repl'response: True\n"]


In [197]:
query = "sort this list of elements alphabetically ['screwdriver', 'pliers', 'hammer']"
response_tools = default_fc_chain.invoke(query)
pprint(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'python_repl',
  'tool_input': {'command': "print(sorted(['screwdriver', 'pliers', "
                            "'hammer']))"}}]
["Tool 'python_repl'response: ['hammer', 'pliers', 'screwdriver']\n"]


### Function Calling pipeline 

In [198]:
function_calling_system_prompt = """you are an helpful assistant and you have access to the following tools:

{tools}

You must always select one or more of the above tools and answer with only a list of JSON objects matching the following schema:

```json
[{{
  "tool": <name of the selected tool>,
  "tool_input": <parameters for the selected tool, matching the tool's JSON schema>
}}]
```

Think step by step
Do not call a tool if the input depends on another tool output you dont have yet
Do not try to answer until you get tools output, if you dont have an answer yet you can continue calling tools

"""

In [199]:
function_calling_chat_template = ChatPromptTemplate.from_messages([('system', function_calling_system_prompt)])
function_calling_chat_template

ChatPromptTemplate(input_variables=['tools'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tools'], template='you are an helpful assistant and you have access to the following tools:\n\n{tools}\n\nYou must always select one or more of the above tools and answer with only a list of JSON objects matching the following schema:\n\n```json\n[{{\n  "tool": <name of the selected tool>,\n  "tool_input": <parameters for the selected tool, matching the tool\'s JSON schema>\n}}]\n```\n\nThink step by step\nDo not call a tool if the input depends on another tool output you dont have yet\nDo not try to answer until you get tools output, if you dont have an answer yet you can continue calling tools\n\n'))])

In [201]:
def msgs_to_llama3_str(msgs: list):
    formatted_msgs = []
    for msg in msgs:
        if msg.type == 'system':
            sys_placeholder = '<|begin_of_text|><|start_header_id|>system<|end_header_id|>system<|end_header_id|> {msg}'
            formatted_msgs.append(sys_placeholder.format(msg=msg.content))
        elif msg.type == 'human':
            human_placeholder = '<|eot_id|><|start_header_id|>user<|end_header_id|>\nUser: {msg} <|eot_id|><|start_header_id|>assistant<|end_header_id|>\nAssistant:'
            formatted_msgs.append(human_placeholder.format(msg=msg.content))
        elif msg.type == 'ai':
            assistant_placeholder = '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\nAssistant: {msg}'
            formatted_msgs.append(assistant_placeholder.format(msg=msg.content))
        elif msg.type == 'tool':
            tool_placeholder = '<|eot_id|><|start_header_id|>tools<|end_header_id|>\n{msg} <|eot_id|><|start_header_id|>assistant<|end_header_id|>\nAssistant:'
            formatted_msgs.append(tool_placeholder.format(msg=msg.content))
    return '\n'.join(formatted_msgs)

In [202]:
def function_call_llm(query, max_it=5, debug=False):
    history = function_calling_chat_template.format_prompt(tools=tools_schemas).to_messages()
    history.append(HumanMessage(query))
    tool_call_id = 0

    for i in range(max_it):
        prompt = msgs_to_llama3_str(history)
        llm_response = llm.invoke(prompt)
        parsed_tools_llm_response = json_parsing_chain.invoke(llm_response)
        history.append(AIMessage(llm_response))
        final_answer, tools_msgs = execute(parsed_tools_llm_response)
        if final_answer:
            final_response = tools_msgs
            if debug:
                pprint(history)
            return final_response
        else:
            history.append(ToolMessage('\n'.join(tools_msgs), tool_call_id=tool_call_id))
            tool_call_id += 1

    raise Exception('not a final response yet', json.dumps(history))

In [203]:
response = function_call_llm('what time is it?', max_it=5, debug=True)

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "title": "Kind",\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression ",\n  "properties": {\n    "expression": {\n      "title": "Expression",\n      "description": "expression to calculate, example \'12 * 3\'",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to execute python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...), if you need a specific module you should import it`.",\n  "p

In [204]:
print(response)

The current date and time is 17/06/2024, 11:36:42.


In [205]:
response = function_call_llm('it is time to go to sleep, how many hours last to 10pm?', max_it=5, debug=True)

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "title": "Kind",\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression ",\n  "properties": {\n    "expression": {\n      "title": "Expression",\n      "description": "expression to calculate, example \'12 * 3\'",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to execute python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...), if you need a specific module you should import it`.",\n  "p

In [206]:
response

'There are 9 hours and 12 minutes left until 10pm.'

In [207]:
response = function_call_llm("is this word is a palindrome? 'saippuakivikauppias'", max_it=5, debug=True)

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "title": "Kind",\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression ",\n  "properties": {\n    "expression": {\n      "title": "Expression",\n      "description": "expression to calculate, example \'12 * 3\'",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to execute python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...), if you need a specific module you should import it`.",\n  "p

In [208]:
response

"Yes, the word'saippuakivikauppias' is a palindrome."

In [209]:
response = function_call_llm(
    "sort this list of elements alphabetically ['screwdriver', 'pliers', 'hammer']", max_it=5, debug=True
)

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "title": "Kind",\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression ",\n  "properties": {\n    "expression": {\n      "title": "Expression",\n      "description": "expression to calculate, example \'12 * 3\'",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to execute python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...), if you need a specific module you should import it`.",\n  "p

In [210]:
response

"The sorted list is: ['hammer', 'pliers','screwdriver']"